In [ ]:
import json
import sklearn
import numpy as np
import pandas as pd

from tqdm import tqdm

from itertools import product
from collections.abc import Iterable

from sklearn.model_selection import (
    StratifiedKFold, 
    StratifiedShuffleSplit
)
from sklearn.metrics import (
    precision_score,
    recall_score,
    accuracy_score,
    roc_auc_score,
    f1_score
)
from sklearn.cross_decomposition import PLSRegression

from imblearn.over_sampling import SMOTE

data_split 함수는 데이터를 분할하는 함수입니다. scikit-learn의 StratifiedShuffleSplit을 사용해 데이터를 분할합니다. 
- StratifiedShuffleSplit은 데이터를 섞은 후에 나누는 것이 아니라, 데이터를 나눈 후에 섞는 방식으로 동작합니다. 


함수는 아래와 같은 과정으로 동작합니다.
1. StratifiedShuffleSplit의 객체, sss를 생성합니다.
- sss는 데이터를 1번 분할하고, 테스트 데이터의 비율을 20%로 설정한다는 정보가 담겨있습니다.

2. sss.split(X, y)를 호출합니다.
- X는 데이터의 feature, y는 데이터의 label입니다.
- split 함수는 X와 y를 test_size 비율로 나눈 후에, train_idx와 test_idx를 반환합니다.
- train_idx와 test_idx는 각각 train 데이터와 test 데이터에 사용할 데이터의 인덱스입니다.

3. train_idx와 test_idx를 사용해 train_x, test_x, train_y, test_y에 해당되는 데이터를 저장합니다.
- reset_index(drop = True)는 인덱스를 0부터 다시 시작하도록 설정함을 의미합니다.

4. train_x, test_x, train_y, test_y를 반환합니다.

In [ ]:
def data_split(X, y, seed):
    sss = StratifiedShuffleSplit(n_splits = 1, test_size = 0.2, random_state = seed)
    
    for train_idx, test_idx in sss.split(X, y):
        train_x = X.iloc[train_idx].reset_index(drop = True)
        train_y = y.iloc[train_idx].reset_index(drop = True)
        test_x = X.iloc[test_idx].reset_index(drop = True)
        test_y = y.iloc[test_idx].reset_index(drop = True)
    
    return train_x, test_x, train_y, test_y

parameter_grid는 하이퍼파라미터의 dictionary를 받아서 가능한 모든 조합의 하이퍼파라미터의 리스트를 생성하고 이를 반환합니다.


함수의 동작 과정은 아래와 같습니다
1. 들어온 입력이 dictionary가 아니면 TypeError를 발생시킵니다.
2. dictionary의 value가 iterable이 아니면 TypeError를 발생시킵니다.
    - iterable이란 list, tuple, set, dict, str 와 같이 맴버들을 한 번에 한 개씩 반환할 수 있는 객체를 말합니다.
    - int, float, bool 등의 객체는 iterable 하지 않습니다.

3. dictionary의 항목들을 key를 기준으로 정렬합니다.
4. dictionary의 key와 value를 각각 keys와 values에 저장합니다.
5. itertools.product를 사용하여 values의 모든 조합을 생성합니다.
6. 생성된 조합을 dictionary로 변환하여 params_grid에 저장합니다.
7. params_grid를 반환합니다.


In [ ]:
def parameter_grid(param_dict):
    if not isinstance(param_dict, dict):
        raise TypeError('Parameter grid is not a dict ({!r})'.format(param_dict))
    
    if isinstance(param_dict, dict):
        for key in param_dict:
            if not isinstance(param_dict[key], Iterable):
                raise TypeError('Parameter grid value is not iterable '
                                '(key={!r}, value={!r})'.format(key, param_dict[key]))
    
    items = sorted(param_dict.items())
    keys, values = zip(*items)
    
    params_grid = []
    for v in product(*values):
        params_grid.append(dict(zip(keys, v))) 
    
    return params_grid

multiclass_cross_validation은 모델과 데이터를 받아서 5-fold cross validation을 수행하고, 각 fold에 대한 평균 precision, recall, f1, accuracy를 계산하여 반환하는 함수입니다.

k-fold cross validation을 수행하기 위해서 StratifiedKFold를 사용합니다. 
- StratifiedKFold는 각 fold에 대해서 클래스의 비율이 유지되도록 데이터를 분할합니다. 

평가를 수행하는 metric은 아래와 같습니다.
1. precision
2. recall
3. f1
4. accuracy

각 fold에 대한 평가를 저장하기 위한 변수(리스트)를 선언합니다.


skf.split(x, y)는 x와 y를 5-fold로 분할한 후, 각 fold에 대한 train index와 validation index를 반환하는 함수입니다.
해당되는 index를 사용하여 train_x, train_y, val_x, val_y를 추출하고, 모델을 학습시킵니다.


PLSRegression은 다중 클래스 분류를 지원하지 않기 때문에, 별도로 one-hot encoding을 수행합니다.
- 예측값을 처리할 때, argmax를 사용하여 가장 높은 확률을 가진 클래스를 선택합니다.


학습된 모델을 사용하여 train_x, val_x에 대한 예측값을 구하고, 각 metric을 계산합니다.
- metric을 계산할 때, average = 'macro'로 설정합니다.
    - average = 'macro'는 클래스별로 metric을 계산한 후, 평균을 계산하는 방식입니다.


반환될 result는 다음과 형태를 가집니다.

{
    'train_precision': 0.95,
    'train_recall': 0.94,
    'train_f1': 0.94,
    'train_accuracy': 0.95,
    'val_precision': 0.92,
    'val_recall': 0.91,
    'val_f1': 0.91,
    'val_accuracy': 0.92
}

In [ ]:
def multiclass_cross_validation(model, x, y, seed):
    skf = StratifiedKFold(n_splits = 5, shuffle = True, random_state = seed)
    
    metrics = ['precision', 'recall', 'f1', 'accuracy']
    
    train_metrics = list(map(lambda x: 'train_' + x, metrics))
    val_metrics = list(map(lambda x: 'val_' + x, metrics))
    
    train_precision = []
    train_recall = []
    train_f1 = []
    train_accuracy = []
    
    val_precision = []
    val_recall = []
    val_f1 = []
    val_accuracy = []
    
    for train_idx, val_idx in skf.split(x, y):
        train_x, train_y = x.iloc[train_idx], y.iloc[train_idx]
        val_x, val_y = x.iloc[val_idx], y.iloc[val_idx]
        
        if type(model) == sklearn.cross_decomposition._pls.PLSRegression:
            onehot_train_y = pd.get_dummies(train_y)
            
            model.fit(train_x, onehot_train_y)
            
            train_pred = np.argmax(model.predict(train_x), axis = 1)
            val_pred = np.argmax(model.predict(val_x), axis = 1)
            
        else:
            model.fit(train_x, train_y)
            
            train_pred = model.predict(train_x)
            val_pred = model.predict(val_x)
        
        train_precision.append(precision_score(train_y, train_pred, average = 'macro'))
        train_recall.append(recall_score(train_y, train_pred, average = 'macro'))
        train_f1.append(f1_score(train_y, train_pred, average = 'macro'))
        train_accuracy.append(accuracy_score(train_y, train_pred))

        val_precision.append(precision_score(val_y, val_pred, average = 'macro'))
        val_recall.append(recall_score(val_y, val_pred, average = 'macro'))
        val_f1.append(f1_score(val_y, val_pred, average = 'macro'))
        val_accuracy.append(accuracy_score(val_y, val_pred))
        
    result = dict(zip(train_metrics + val_metrics, 
                      [np.mean(train_precision), np.mean(train_recall), np.mean(train_f1), np.mean(train_accuracy), 
                       np.mean(val_precision), np.mean(val_recall), np.mean(val_f1), np.mean(val_accuracy)]))
    
    return(result)

In [ ]:
def binary_cross_validation(model, x, y, seed):
    skf = StratifiedKFold(n_splits = 5, shuffle = True, random_state = seed)
    
    metrics = ['precision', 'recall', 'f1', 'accuracy', 'auc']
    
    train_metrics = list(map(lambda x: 'train_' + x, metrics))
    val_metrics = list(map(lambda x: 'val_' + x, metrics))
    
    train_precision = []
    train_recall = []
    train_f1 = []
    train_accuracy = []
    train_auc = []
    
    val_precision = []
    val_recall = []
    val_f1 = []
    val_accuracy = []
    val_auc = []
    
    for train_idx, val_idx in skf.split(x, y):
        train_x, train_y = x.iloc[train_idx], y.iloc[train_idx]
        val_x, val_y = x.iloc[val_idx], y.iloc[val_idx]
        
        if type(model) == sklearn.cross_decomposition._pls.PLSRegression:
            model.fit(train_x, train_y)
            
            train_pred_score = model.predict(train_x)
            train_pred = np.where(train_pred_score < 0.5, 0, 1).reshape(-1)
            
            val_pred_score = model.predict(val_x)
            val_pred = np.where(val_pred_score < 0.5, 0, 1).reshape(-1)
            
        else:
            model.fit(train_x, train_y)
            
            train_pred = model.predict(train_x)
            train_pred_score = model.predict_proba(train_x)[:, 1]
            
            val_pred = model.predict(val_x)
            val_pred_score = model.predict_proba(val_x)[:, 1]
        
        train_precision.append(precision_score(train_y, train_pred))
        train_recall.append(recall_score(train_y, train_pred))
        train_f1.append(f1_score(train_y, train_pred))
        train_accuracy.append(accuracy_score(train_y, train_pred))
        train_auc.append(roc_auc_score(train_y, train_pred_score))

        val_precision.append(precision_score(val_y, val_pred))
        val_recall.append(recall_score(val_y, val_pred))
        val_f1.append(f1_score(val_y, val_pred))
        val_accuracy.append(accuracy_score(val_y, val_pred))
        val_auc.append(roc_auc_score(val_y, val_pred_score))

    result = dict(zip(train_metrics + val_metrics, 
                      [np.mean(train_precision), np.mean(train_recall), np.mean(train_f1), np.mean(train_accuracy), np.mean(train_auc), 
                       np.mean(val_precision), np.mean(val_recall), np.mean(val_f1), np.mean(val_accuracy), np.mean(val_auc)]))
    
    return(result)

In [ ]:
def binary_smote_cross_validation(model, x, y, seed, args):
    skf = StratifiedKFold(n_splits = 5, shuffle = True, random_state = seed)
    
    metrics = ['precision', 'recall', 'f1', 'accuracy', 'auc']
    
    train_metrics = list(map(lambda x: 'train_' + x, metrics))
    val_metrics = list(map(lambda x: 'val_' + x, metrics))
    
    train_precision = []
    train_recall = []
    train_f1 = []
    train_accuracy = []
    train_auc = []
    
    val_precision = []
    val_recall = []
    val_f1 = []
    val_accuracy = []
    val_auc = []
    
    for train_idx, val_idx in skf.split(x, y):
        train_x, train_y = x.iloc[train_idx], y.iloc[train_idx]
        val_x, val_y = x.iloc[val_idx], y.iloc[val_idx]
        
        smote = SMOTE(random_state = args.smoteseed, k_neighbors = args.neighbor)
        train_x, train_y = smote.fit_resample(train_x, train_y)
        
        if type(model) == sklearn.cross_decomposition._pls.PLSRegression:
            model.fit(train_x, train_y)
            
            train_pred_score = model.predict(train_x)
            train_pred = np.where(train_pred_score < 0.5, 0, 1).reshape(-1)
            
            val_pred_score = model.predict(val_x)
            val_pred = np.where(val_pred_score < 0.5, 0, 1).reshape(-1)
            
        else:
            model.fit(train_x, train_y)
            
            train_pred = model.predict(train_x)
            train_pred_score = model.predict_proba(train_x)[:, 1]
            
            val_pred = model.predict(val_x)
            val_pred_score = model.predict_proba(val_x)[:, 1]
        
        train_precision.append(precision_score(train_y, train_pred))
        train_recall.append(recall_score(train_y, train_pred))
        train_f1.append(f1_score(train_y, train_pred))
        train_accuracy.append(accuracy_score(train_y, train_pred))
        train_auc.append(roc_auc_score(train_y, train_pred_score))

        val_precision.append(precision_score(val_y, val_pred))
        val_recall.append(recall_score(val_y, val_pred))
        val_f1.append(f1_score(val_y, val_pred))
        val_accuracy.append(accuracy_score(val_y, val_pred))
        val_auc.append(roc_auc_score(val_y, val_pred_score))

    result = dict(zip(train_metrics + val_metrics, 
                      [np.mean(train_precision), np.mean(train_recall), np.mean(train_f1), np.mean(train_accuracy), np.mean(train_auc), 
                       np.mean(val_precision), np.mean(val_recall), np.mean(val_f1), np.mean(val_accuracy), np.mean(val_auc)]))
    
    return(result)

사용되지 않는 함수입니다.

In [ ]:
# def CV(x, y, model, params, seed):
#     skf = StratifiedKFold(n_splits = 5, shuffle = True, random_state = seed)
    
#     metrics = ['precision', 'recall', 'f1', 'accuracy']
    
#     train_metrics = list(map(lambda x: 'train_' + x, metrics))
#     val_metrics = list(map(lambda x: 'val_' + x, metrics))
    
#     train_precision = []
#     train_recall = []
#     train_f1 = []
#     train_accuracy = []
    
#     val_precision = []
#     val_recall = []
#     val_f1 = []
#     val_accuracy = []
    
#     for train_idx, val_idx in skf.split(x, y):
#         train_x, train_y = x.iloc[train_idx], y.iloc[train_idx]
#         val_x, val_y = x.iloc[val_idx], y.iloc[val_idx]
        
#         try:
#             clf = model(random_state = seed, **params)
#         except:
#             clf = model(**params)
        
        
#         if model == sklearn.cross_decomposition._pls.PLSRegression:
#             onehot_train_y = pd.get_dummies(train_y)
            
#             clf.fit(train_x, onehot_train_y)
            
#             train_pred = np.argmax(clf.predict(train_x), axis = 1)
#             val_pred = np.argmax(clf.predict(val_x), axis = 1)
            
#         else:
#             clf.fit(train_x, train_y)
            
#             train_pred = clf.predict(train_x)
#             val_pred = clf.predict(val_x)
        
#         train_precision.append(precision_score(train_y, train_pred, average = 'macro'))
#         train_recall.append(recall_score(train_y, train_pred, average = 'macro'))
#         train_f1.append(f1_score(train_y, train_pred, average = 'macro'))
#         train_accuracy.append(accuracy_score(train_y, train_pred))

#         val_precision.append(precision_score(val_y, val_pred, average = 'macro'))
#         val_recall.append(recall_score(val_y, val_pred, average = 'macro'))
#         val_f1.append(f1_score(val_y, val_pred, average = 'macro'))
#         val_accuracy.append(accuracy_score(val_y, val_pred))
        
#     result = dict(zip(['params'] + train_metrics + val_metrics, 
#                       [params] + [np.mean(train_precision), 
#                                   np.mean(train_recall), 
#                                   np.mean(train_f1), 
#                                   np.mean(train_accuracy), 
#                                   np.mean(val_precision), 
#                                   np.mean(val_recall), 
#                                   np.mean(val_f1), 
#                                   np.mean(val_accuracy)]))
    
#     return(result)

In [ ]:
def metric_mean(data, metric: str):
    mean_per_hp = list(map(lambda x: np.mean(x[1]), data[metric].items()))
    return mean_per_hp

print_best_param은 val_result에서 metric에 해당하는 평균값이 가장 높은 파라미터를 반환하는 함수입니다.


함수는 아래의 과정으로 동작합니다.
1. val_result에서 metric에 해당하는 평균값을 구합니다.
2. 평균값이 가장 높은 파라미터의 인덱스를 구합니다.
3. val_result에서 해당 인덱스에 해당하는 파라미터를 반환합니다.

In [ ]:
def print_best_param(val_result, metric: str):
    
    mean_list = metric_mean(val_result, metric)
    max_idx = mean_list.index(max(mean_list))
    
    best_param = val_result['model'][f'model{max_idx}']
    
    return best_param

In [ ]:
def load_val_result(path: str, tg_num: int, inhale_type: str, model: str, is_smote = True):
    if is_smote:
        try:
            with open(f'{path}/tg{tg_num}_val_results/binary_smote5/{inhale_type}_{model}.json', 'r') as file:
                val_result = json.load(file)
        except:
            with open(f'{path}tg{tg_num}_val_results/binary_smote5/{inhale_type}_{model}.json', 'r') as file:
                val_result = json.load(file)
    else:
        try:
            with open(f'{path}/tg{tg_num}_val_results/binary/{inhale_type}_{model}.json', 'r') as file:
                val_result = json.load(file)
        except:
            with open(f'{path}tg{tg_num}_val_results/binary/{inhale_type}_{model}.json', 'r') as file:
                val_result = json.load(file)
    
    return val_result


사용되지 않는 함수입니다.

In [ ]:
# def print_best_param(path: str, tg_num: int, inhale_type: str, model: str, metric: str):
#     data = load_val_result(path, tg_num, inhale_type, model)
    
#     mean_list = metric_mean(data, 'f1')
#     max_idx = mean_list.index(max(mean_list))
    
#     best_param = data['model'][f'model{max_idx}']
    
#     return best_param